In [34]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Standard
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from PIL import Image
import scipy

# Tensorflow and Keras
from keras.datasets import mnist
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
import tensorflow as tf

from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

os.chdir('../')
from src import image_preprocess as ip

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Start with histology data
for each 40X, 100X, 200X, 400X, have 644 items for benign and 1300 for malignant

In [28]:
# Data is already fairly uniform. No major changes for augmentation
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True,
                            fill_mode='constant',
                            cval=0)

In [29]:
img = Image.open('../data/Histology/100X/benign/SOB_B_A-14-22549AB-100-001.png')

In [30]:
x = img_to_array(img)
print(x.shape)
x = x.reshape((1,) + x.shape)
print(x.shape)

(460, 700, 3)
(1, 460, 700, 3)


In [31]:
import os
os.getcwd()

'/home/maureen/Documents/Galvanize/Capstone1/Capstone3/Cancer_Prediction/notebooks'

In [32]:
# Create new images
i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='new_benign',
                         save_prefix='benign',
                         save_format='jpeg'):
    i += 1 
    if i > 5:
        break

In [33]:
## For Mammograms

In [ ]:
# Data is already fairly uniform. No major changes for augmentation
datagen = ImageDataGenerator(rotation_range=5,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True,
                            fill_mode='constant',
                            cval=0)